<a href="https://colab.research.google.com/github/ab2gbl/Master2/blob/main/BDPA/TP/BDPA_tp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.appName("sparkFirstLab").getOrCreate()
sc = SparkContext.getOrCreate()

In [ ]:
df = spark.read.csv('./epl-training.csv', header=True)

In [ ]:
list(df.columns)

['Date',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR',
 'HTHG',
 'HTAG',
 'HTR',
 'Referee',
 'HS',
 'AS',
 'HST',
 'AST',
 'HC',
 'AC',
 'HF',
 'AF',
 'HY',
 'AY',
 'HR',
 'AR']

In [ ]:
df.show(5)

+----------+--------+-------------+----+----+---+----+----+---+----------------+---+---+---+---+---+---+---+---+---+---+---+---+
|      Date|HomeTeam|     AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|         Referee| HS| AS|HST|AST| HC| AC| HF| AF| HY| AY| HR| AR|
+----------+--------+-------------+----+----+---+----+----+---+----------------+---+---+---+---+---+---+---+---+---+---+---+---+
|19/08/2000|Charlton|     Man City|   4|   0|  H|   2|   0|  H|      Rob Harris| 17|  8| 14|  4|  6|  6| 13| 12|  1|  2|  0|  0|
|19/08/2000| Chelsea|     West Ham|   4|   2|  H|   1|   0|  H|   Graham Barber| 17| 12| 10|  5|  7|  7| 19| 14|  1|  2|  0|  0|
|19/08/2000|Coventry|Middlesbrough|   1|   3|  A|   1|   1|  D|    Barry Knight|  6| 16|  3|  9|  8|  4| 15| 21|  5|  3|  1|  0|
|19/08/2000|   Derby|  Southampton|   2|   2|  D|   1|   2|  A|     Andy D'Urso|  6| 13|  4|  6|  5|  8| 11| 13|  1|  1|  0|  0|
|19/08/2000|   Leeds|      Everton|   2|   0|  H|   2|   0|  H|Dermot Gallagher| 17| 12|  8|  6| 

In [ ]:
teams = df.select("HomeTeam").distinct()
teams.show()


+----------------+
|        HomeTeam|
+----------------+
|       Tottenham|
|        Brighton|
|          Bolton|
|      Sunderland|
|   Nott'm Forest|
|        Bradford|
|            Hull|
|        Coventry|
|         Arsenal|
|        Man City|
|       Brentford|
|       Blackburn|
|         Cardiff|
|Sheffield United|
|         Norwich|
|  Crystal Palace|
|      Birmingham|
|         Burnley|
|           Derby|
|       Leicester|
+----------------+
only showing top 20 rows



In [ ]:
df.groupBy("HomeTeam").agg({"FTHG": "sum"}) \
  .orderBy("sum(FTHG)", ascending=False) \
  .show(5)


+----------+---------+
|  HomeTeam|sum(FTHG)|
+----------+---------+
|   Arsenal|   1008.0|
|  Man City|    966.0|
| Liverpool|    955.0|
|Man United|    948.0|
|   Chelsea|    942.0|
+----------+---------+
only showing top 5 rows



In [ ]:
df.groupBy("AwayTeam").agg({"FTHG": "sum"}) \
  .orderBy("sum(FTHG)", ascending=False) \
  .show(5)


+-----------+---------+
|   AwayTeam|sum(FTHG)|
+-----------+---------+
|  Newcastle|    704.0|
|   West Ham|    675.0|
|    Everton|    672.0|
|  Tottenham|    665.0|
|Aston Villa|    650.0|
+-----------+---------+
only showing top 5 rows



In [ ]:
home_goals = df.groupBy("HomeTeam").agg({"FTHG": "sum"}).withColumnRenamed("sum(FTHG)", "HomeGoals")
away_goals = df.groupBy("AwayTeam").agg({"FTAG": "sum"}).withColumnRenamed("sum(FTAG)", "AwayGoals")

total_goals = home_goals.join(away_goals, home_goals.HomeTeam == away_goals.AwayTeam) \
                        .withColumn("TotalGoals", home_goals.HomeGoals + away_goals.AwayGoals) \
                        .select("HomeTeam", "TotalGoals") \
                        .orderBy("TotalGoals", ascending=False)

total_goals.show(3)


+---------+----------+
| HomeTeam|TotalGoals|
+---------+----------+
|  Arsenal|    1763.0|
|Liverpool|    1700.0|
| Man City|    1690.0|
+---------+----------+
only showing top 3 rows



In [ ]:
home_matches = df.groupBy("HomeTeam").count().withColumnRenamed("count", "HomeMatches")
away_matches = df.groupBy("AwayTeam").count().withColumnRenamed("count", "AwayMatches")

total_matches = home_matches.join(away_matches, home_matches.HomeTeam == away_matches.AwayTeam) \
                            .withColumn("TotalMatches", home_matches.HomeMatches + away_matches.AwayMatches) \
                            .select("HomeTeam", "TotalMatches") \
                            .orderBy("TotalMatches", ascending=False)

total_matches.show(1)


+---------+------------+
| HomeTeam|TotalMatches|
+---------+------------+
|Tottenham|         922|
+---------+------------+
only showing top 1 row



In [ ]:
from pyspark.sql.functions import to_date, year, col

# Convert the 'Date' column to a proper DateType
df = df.withColumn("Date", to_date(col("Date"), "dd/MM/yyyy"))

# Extract the year from the Date column
df = df.withColumn("Year", year(col("Date")))

# Calculate the current year
current_year = 2024  # Replace with `datetime.now().year` if using dynamically

# Filter the dataset for matches played in the last decade
last_decade_df = df.filter(col("Year") >= current_year - 10)

In [ ]:
home_goals = last_decade_df.groupBy("HomeTeam").agg({"FTHG": "sum"}).withColumnRenamed("sum(FTHG)", "HomeGoals")
away_goals = last_decade_df.groupBy("AwayTeam").agg({"FTAG": "sum"}).withColumnRenamed("sum(FTAG)", "AwayGoals")
home_conceded = last_decade_df.groupBy("HomeTeam").agg({"FTAG": "sum"}).withColumnRenamed("sum(FTAG)", "HomeConceded")
away_conceded = last_decade_df.groupBy("AwayTeam").agg({"FTHG": "sum"}).withColumnRenamed("sum(FTHG)", "AwayConceded")

total_goals = home_goals.join(away_goals, home_goals.HomeTeam == away_goals.AwayTeam) \
                        .withColumn("TotalGoals", home_goals.HomeGoals + away_goals.AwayGoals) \
                        .select("HomeTeam", "TotalGoals") \
                        .orderBy("TotalGoals", ascending=False)

total_conceded = home_conceded.join(away_conceded, home_conceded.HomeTeam == away_conceded.AwayTeam) \
                        .withColumn("TotalConceded", home_conceded.HomeConceded + away_conceded.AwayConceded) \
                        .select("HomeTeam", "TotalConceded") \
                        .orderBy("TotalConceded", ascending=False)

ratios = total_goals.join(total_conceded, total_goals.HomeTeam == total_conceded.HomeTeam).withColumn('ratio',total_goals.TotalGoals/total_conceded.TotalConceded).orderBy("ratio", ascending=False)
ratios.show(5)


+---------+----------+---------+-------------+------------------+
| HomeTeam|TotalGoals| HomeTeam|TotalConceded|             ratio|
+---------+----------+---------+-------------+------------------+
| Man City|     979.0| Man City|        351.0|2.7891737891737893|
|Liverpool|     854.0|Liverpool|        425.0|2.0094117647058822|
|  Arsenal|     765.0|  Arsenal|        456.0|1.6776315789473684|
|  Chelsea|     709.0|  Chelsea|        447.0|1.5861297539149888|
|Tottenham|     751.0|Tottenham|        481.0|1.5613305613305613|
+---------+----------+---------+-------------+------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import avg, month

# Add month column
df = df.withColumn("Month", month(col("Date")))

# Define periods
periods = [
    (9, 11, "Sep-Nov"),
    (12, 3, "Dec-Mar"),
    (4, 6, "Apr-Jun")
]

for start, end, label in periods:
    period_df = df.filter((col("Month") >= start%12) & (col("Month") <= end%12))
    avg_goals = period_df.select((col("FTHG") + col("FTAG")).alias("TotalGoals")).agg(avg("TotalGoals").alias("AvgGoals"))
    print(f"Period: {label}")
    avg_goals.show()

Period: Sep-Nov
+------------------+
|          AvgGoals|
+------------------+
|2.7249426166794186|
+------------------+

Period: Dec-Mar
+------------------+
|          AvgGoals|
+------------------+
|2.6525066972828166|
+------------------+

Period: Apr-Jun
+------------------+
|          AvgGoals|
+------------------+
|2.7919203099059215|
+------------------+

